In [1]:
#!pip install pydlm

In [2]:
from pydlm import dlm, trend, seasonality, dynamic, autoReg, longSeason
import pandas as pd

In [5]:
df = pd.read_csv('mv_port_processed.csv')

In [6]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Project Number'] = df['Project Number'].astype('str') 
df = df.loc[:, ~df.columns.str.contains('^index')]

In [7]:
df_sub = df[['Date','Project Number','Actual Net Revenue - MTD','Project Status', 'Contract Type Name', 'Ultimate Customer AUS Market Sector', 'Division Short Name', 'Proj_CVP', 'Proj_Serv_Offering',
             'primary_client_industry', 'Funding Amount - ITC (Snapshot)', 'Backlog NR',  'Actual Direct Labor Cost - MTD', 'Actual Non-Labor Cost - MTD',
       'Actual Gross Revenue - MTD', 'EBITA - MTD',
       'Forecast Direct Labor Cost - ITC', 'Forecast Net Revenue - ITC',
       'Order Intake (NR) - MTD', 'AUS Mkt Sector Aggregate', 'Baseline NR', 'Organization Size',
       'Risk Profile', 'riskrating_project', 'ActualNR_sum', 'Projectduration', 'Completion', 'scheduled_duration', 'Scheduled_projectduration', 'intime', 'seasonality_month']]

In [8]:
len(df_sub)

1269171

In [ ]:
df_sub.groupby(['Date'])['Actual Net Revenue - MTD']

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor() 
gbr.fit(X_train, y_train)

from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()
forest.fit(X_train, y_train)

import xgboost as xgb

xgbr = xgb.XGBRegressor()
xgbr.fit(X_train, y_train)

### We need to dummy encode now

In [8]:
Project_Status = pd.get_dummies(df_sub['Project Status'], drop_first=True)
Contract_Name = pd.get_dummies(df_sub['Contract Type Name'], drop_first=True)
Ultimate_AUS = pd.get_dummies(df_sub['Ultimate Customer AUS Market Sector'], drop_first=True)
Divison = pd.get_dummies(df_sub['Division Short Name'], drop_first=True)
#Region = pd.get_dummies(df_sub['Region'], drop_first=True)
Proj_CVP = pd.get_dummies(df_sub['Proj_CVP'], drop_first=True)
Proj_Serv_Offering = pd.get_dummies(df_sub['Proj_Serv_Offering'], drop_first=True)
Client = pd.get_dummies(df_sub['primary_client_industry'], drop_first=True)
Aggregate_AUS = pd.get_dummies(df_sub['AUS Mkt Sector Aggregate'], drop_first=True)
Orgsize = pd.get_dummies(df_sub['Organization Size'], drop_first=True)
Profile = pd.get_dummies(df_sub['Risk Profile'], drop_first=True)



df_sub = pd.concat([df_sub, Project_Status], axis=1)
df_sub = pd.concat([df_sub, Contract_Name], axis=1)
df_sub = pd.concat([df_sub, Ultimate_AUS], axis=1)
df_sub = pd.concat([df_sub, Divison], axis=1)
#df_sub = pd.concat([df_sub, Region], axis=1)
df_sub = pd.concat([df_sub, Proj_CVP], axis=1)
df_sub = pd.concat([df_sub, Proj_Serv_Offering], axis=1)
df_sub = pd.concat([df_sub, Client], axis=1)
df_sub = pd.concat([df_sub, Aggregate_AUS], axis=1)
df_sub = pd.concat([df_sub, Orgsize], axis=1)
df_sub = pd.concat([df_sub, Profile], axis=1)

df_sub = df_sub.drop(columns=['Project Status', 'Contract Type Name', 'Ultimate Customer AUS Market Sector', 'Division Short Name',
                              'Proj_CVP', 'Proj_Serv_Offering', 'primary_client_industry', 'AUS Mkt Sector Aggregate', 'Organization Size', 'Risk Profile'])
df_sub

,Date,Project Number,Actual Net Revenue - MTD,Funding Amount - ITC (Snapshot),Backlog NR,Actual Direct Labor Cost - MTD,Actual Non-Labor Cost - MTD,Actual Gross Revenue - MTD,EBITA - MTD,Forecast Direct Labor Cost - ITC,...,Transp/Log,Key Global,Key Local,Key Regional,Local,RKC,Unspecified,Low,Medium,Unspecified
0,201711,000221760018,111.72,68313,0.00,46.65,NaN,111.72,-5.17,0.00,...,0,0,0,0,0,1,0,0,0,1
1,201712,000221760018,10406.45,68313,0.00,1353.76,NaN,52149.10,7014.47,0.00,...,0,0,0,0,0,1,0,0,0,1
2,201801,000221760018,-2048.91,68313,0.00,17.55,NaN,-2048.91,-2092.03,0.00,...,0,0,0,0,0,1,0,0,0,1
3,201802,000221760018,87.02,68313,3207.95,36.55,NaN,14893.44,-2.78,2064.61,...,0,0,0,0,0,1,0,0,0,1
4,201803,000221760018,185.22,68313,3022.73,85.81,NaN,185.22,-25.60,2064.61,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269166,201902,zz000zzznt20,0.00,0,0.00,0.00,NaN,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,1
1269167,201903,zz000zzznt20,0.00,0,0.00,0.00,NaN,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,1
1269168,201904,zz000zzznt20,0.00,0,0.00,0.00,NaN,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,1
1269169,201905,zz000zzznt20,0.00,0,0.00,0.00,NaN,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,1


In [11]:
correlation_df = df[['Actual Net Revenue - MTD', 'Funding Amount - ITC (Snapshot)', 'Backlog NR', 'Actual Direct Labor Cost - MTD', 'Actual Non-Labor Cost - MTD', 'Actual Non-Labor Cost - MTD', 'EBITA - MTD', 'Forecast Direct Labor Cost - ITC', 
                    'Forecast Net Revenue - ITC', 'Order Intake (NR) - MTD', 'Baseline NR', 'riskrating_project', 'ActualNR_sum', 'Projectduration', 'Completion', 'Scheduled_projectduration', 'seasonality_month']]


In [19]:
df_sub.corr(method ='pearson')

,Funding Amount - ITC (Snapshot),Backlog NR,Actual Direct Labor Cost - MTD,Actual Non-Labor Cost - MTD,Actual Gross Revenue - MTD,Actual Net Revenue - MTD,EBITA - MTD,Forecast Direct Labor Cost - ITC,Forecast Net Revenue - ITC,Order Intake (NR) - MTD,...,Transp/Log,Key Global,Key Local,Key Regional,Local,RKC,Unspecified,Low,Medium,Unspecified
Funding Amount - ITC (Snapshot),1.000000,0.438156,0.019451,0.373201,0.060993,0.020088,0.021686,0.722600,0.746455,0.078865,...,-0.029490,-0.008763,-0.000166,0.001705,-0.013314,0.051806,-0.010906,-0.045112,0.053158,-0.056150
Backlog NR,0.438156,1.000000,0.021418,0.186730,0.037168,0.020349,0.011202,0.514491,0.532115,0.201932,...,-0.018655,-0.010816,-0.003163,-0.004490,0.000298,0.040833,-0.015491,-0.023658,0.052349,-0.036174
Actual Direct Labor Cost - MTD,0.019451,0.021418,1.000000,0.359079,0.988012,0.993394,0.645172,0.023820,0.023433,0.006276,...,-0.000979,-0.000693,-0.000114,0.000391,0.000009,0.002342,-0.001480,-0.000620,0.003996,-0.001715
Actual Non-Labor Cost - MTD,0.373201,0.186730,0.359079,1.000000,0.770852,0.158402,-0.006945,0.149755,0.153377,0.039923,...,-0.004912,-0.005098,-0.001673,0.010147,-0.002441,0.004993,0.004484,-0.020732,0.012788,-0.002610
Actual Gross Revenue - MTD,0.060993,0.037168,0.988012,0.770852,1.000000,0.994429,0.722923,0.039366,0.041204,0.040976,...,-0.001402,-0.001072,-0.000223,0.001263,-0.000046,0.004166,-0.002799,-0.000607,0.006174,-0.003328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RKC,0.051806,0.040833,0.002342,0.004993,0.004166,0.002374,0.001805,0.063419,0.066124,0.004837,...,0.230834,-0.072962,-0.028525,-0.085375,-0.353903,1.000000,-0.292005,0.066210,-0.016663,0.114710
Unspecified,-0.010906,-0.015491,-0.001480,0.004484,-0.002799,-0.001467,-0.000933,-0.021259,-0.017914,0.002381,...,-0.016468,-0.087353,-0.034151,-0.102215,-0.423708,-0.292005,1.000000,-0.144381,-0.059060,-0.300722
Low,-0.045112,-0.023658,-0.000620,-0.020732,-0.000607,-0.000608,-0.000143,-0.057229,-0.062760,-0.001619,...,0.084702,-0.001940,0.009343,0.087572,-0.035269,0.066210,-0.144381,1.000000,-0.429314,-0.414702
Medium,0.053158,0.052349,0.003996,0.012788,0.006174,0.004069,0.002707,0.073793,0.080322,0.014984,...,0.047165,0.070268,0.036376,0.047724,0.043817,-0.016663,-0.059060,-0.429314,1.000000,-0.269732


In [31]:
len(df_sub.columns.to_list())

241

In [9]:
def obtain_df(project, df):
    df_project = df[df['Project Number'] == project]
    #df_project = df_project[['Fiscal Period Code', 'Funding Amount - ITC (Snapshot)', 'Backlog NR', 'Actual Gross Revenue - MTD']].copy()
    df_project['Backlog_Shifted'] = df_project['Backlog NR'].transform(lambda x: x.shift())
    df_project.reset_index(inplace=True)
    df_project = df_project.loc[:, ~df_project.columns.str.contains('^index')]
    return df_project

def random_project(df):
    df = df.sample(n=1)
    project_number = df['Project Number'].item()
    return project_number

In [94]:
random_project(df_sub)

'30013277'

In [95]:
obtain_df('30013277', df_sub)

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Date,Project Number,Actual Net Revenue - MTD,Funding Amount - ITC (Snapshot),Backlog NR,Actual Direct Labor Cost - MTD,Actual Non-Labor Cost - MTD,Actual Gross Revenue - MTD,EBITA - MTD,Forecast Direct Labor Cost - ITC,...,Key Global,Key Local,Key Regional,Local,RKC,Unspecified,Low,Medium,Unspecified,Backlog_Shifted
0,201604,30013277,1128.40,64250,14851.60,364.00,NaN,1128.40,187.5700,5153.94,...,0,0,0,0,1,0,1,0,0,NaN
1,201605,30013277,6226.13,64250,8625.47,2008.43,NaN,12391.93,1034.9400,5153.94,...,0,0,0,0,1,0,1,0,0,14851.60
2,201606,30013277,6779.89,64250,1845.58,2187.06,NaN,19786.39,1127.0000,5153.94,...,0,0,0,0,1,0,1,0,0,8625.47
3,201607,30013277,3925.99,64250,0.00,1266.45,NaN,18928.17,652.6000,5153.94,...,0,0,0,0,1,0,1,0,0,1845.58
4,201608,30013277,43.96,64250,7103.81,14.18,NaN,4911.30,7.3100,9546.70,...,0,0,0,0,1,0,1,0,0,0.00
5,201609,30013277,363.20,64250,6740.61,117.16,NaN,363.20,60.3800,9546.70,...,0,0,0,0,1,0,1,0,0,7103.81
6,201610,30013277,0.00,64250,5540.61,0.00,NaN,1200.00,0.0000,9546.70,...,0,0,0,0,1,0,1,0,0,6740.61
7,201611,30013277,0.00,64250,5540.61,0.00,NaN,0.00,0.0000,9546.70,...,0,0,0,0,1,0,1,0,0,5540.61
8,201612,30013277,0.00,64250,5540.61,0.00,NaN,0.00,0.0000,7752.74,...,0,0,0,0,1,0,1,0,0,5540.61
9,201701,30013277,0.00,64250,5540.61,0.00,NaN,0.00,0.0000,7752.74,...,0,0,0,0,1,0,1,0,0,5540.61


In [ ]:
# define base model
model = LinearSVR()
# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)

In [ ]:
# construct the base
myDLM = dlm(data)
# adding model components
# add a first-order trend (linear trending) with prior covariance 1.0
myDLM = myDLM + trend(1, name='lineTrend', w=1.0)
# add a 7 day seasonality with prior covariance 1.0
myDLM = myDLM + seasonality(7, name='7day', w=1.0)
# add a 3 step auto regression
myDLM = myDLM + autoReg(degree=3, data=data, name='ar3', w=1.0)
# show the added components
myDLM.ls()
# delete unwanted component
myDLM.delete('7day')
myDLM.ls()

In [ ]:
# fit forward filter
myDLM.fitForwardFilter()
# fit backward smoother
myDLM.fitBackwardSmoother()

In [ ]:
# plot the results
myDLM.plot()
# plot only the filtered results
myDLM.turnOff('smoothed plot')
myDLM.plot()
# plot in one figure
myDLM.turnOff('multiple plots')
myDLM.plot()

In [ ]:
# plot the latent states of the 'ar3'
myDLM.plotCoef(name='ar3')

In [ ]:
from pydlm import modelTuner
myTuner = modelTuner(method='gradient_descent', loss='mse')
tunedDLM = myTuner.tune(myDLM, maxit=100)

In [ ]:
myDLM_mse = myDLM.getMSE()
tunedDLM.fit()
tunedDLM_mse = tunedDLM.getMSE()

In [ ]:
# get the filtered and smoothed results
filteredMean = myDLM.getMean(filterType='forwardFilter')
smoothedMean = myDLM.getMean(filterType='backwardSmoother')
filteredVar = myDLM.getVar(filterType='forwardFilter')
smoothedVar = myDLM.getVar(filterType='backwardSmoother')

filteredCI = myDLM.getInterval(filterType='forwardFilter')
smoothedCI = myDLM.getInterval(filterType='backwardSmoother')

# get the residual time series
residual = myDLM.getResidual(filterType='backwardSmoother')

# get the filtered and smoothed mean for a given component
filteredTrend = myDLM.getMean(filterType='forwardFilter', name='lineTrend')
smoothedTrend = myDLM.getMean(filterType='backwardSmoother', name='lineTrend')

# get the latent states
allStates = myDLM.getLatentState(filterType='forwardFilter')
trendStates = myDLM.getLatentState(filterType='forwardFilter', name='lineTrend')

https://stackoverflow.com/questions/61390630/statsmodels-sarimax-with-exogenous-variables-how-to-extract-exogenous-coeffici

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [43]:
sample2 = obtain_df('30048008', df_sub)

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [67]:
Y.shape

(15,)

In [68]:
X.shape

(15, 240)

In [69]:
import numpy as np

In [71]:
Y

array([   0.  ,    0.  ,    0.  , 6922.19, 4079.42, 3789.8 ,  547.5 ,
        309.81,   71.45,   35.72,   11.91, 1816.15,    0.  ,    0.  ,
         68.91])

### Now we split the series

In [73]:
X_train

array([[0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 1.8313e+02, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [3.3204e+04, 0.0000e+00, 4.2000e+01, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.3204e+04, 0.0000e+00, 2.1000e+01, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.3204e+04, 0.0000e+00, 7.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00]])

[1.81606372e+03 8.17385382e-03 8.17385382e-03 6.92322949e+01]


/Users/julia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


In [88]:
Y_test

array([1816.15,    0.  ,    0.  ,   68.91])

### Using the Sktime model

In [75]:
#!pip install sktime

In [59]:
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting.forecasting import plot_ys

ModuleNotFoundError: No module named 'sktime.utils.plotting.forecasting'; 'sktime.utils.plotting' is not a package

In [ ]:



y = load_airline()
y_train, y_test = temporal_train_test_split(y)
plot_ys(y_train, y_test, labels=["y_train", "y_test"])